In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from os import listdir 
import numpy as np 
from PIL import Image 
import tensorflow as tf 
import keras 
from keras.applications.vgg19 import VGG19 
from sklearn.model_selection import train_test_split 
from keras.models import load_model 

In [ ]:
car_path = '/content/drive/My Drive/CV992/car/'
truck_path = '/content/drive/My Drive/CV992/truck/'
car_img, truck_img = [], []

In [ ]:

for i, each in enumerate(listdir(car_path)):
    try:
        img= Image.open(car_path + each)
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            car_img.append(img)
    except:
        print("ignored loading", each)
print(len(car_img), 'car done')


for i, each in enumerate(listdir(truck_path)):
    try:
        img= Image.open(truck_path + each)
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            truck_img.append(img)
    except:
        print("ignored loading", each)
print(len(truck_img), 'truck done')

ignored loading 181535.jpg
392 car done
396 truck done


In [ ]:
X = car_img + truck_img 
y = [0] * len(car_img) + [1] * len(truck_img) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) 
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, shuffle=True) 
print(len(X_train), len(y_train), len(X_val), len(y_val), len(X_test), len(y_test))

630 630 79 79 79 79


In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)
print('Train :', X_train.shape, y_train.shape) 
print('Val :', X_val.shape, y_val.shape)
print('Test :', X_test.shape, y_test.shape)

Train : (630, 224, 224, 3) (630,)
Val : (79, 224, 224, 3) (79,)
Test : (79, 224, 224, 3) (79,)


In [ ]:
model = VGG19(include_top=False, weights='imagenet', input_shape=(224, 224, 3)) 
for each_layer in model.layers: 
    each_layer.trainable = False
interaction_output = model.layers[-1].output 
x = keras.layers.Flatten()(interaction_output) 
x = keras.layers.Dense(1024, activation="relu")(x) 
x = keras.layers.Dense(1, activation="sigmoid")(x) 
model = keras.Model(model.input, x) 
model.summary() 

80142336/80134624 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint("/content/drive/My Drive/CV992/model_h.h5", monitor='val_acc', save_best_only=True, mode='max')
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy']) 
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size = 16, callbacks=[checkpoint])
model.save('/content/drive/My Drive/CV992/model_h.h5')

Epoch 1/5
40/40 [==============================] - 4s 96ms/step - loss: 2.7373e-04 - accuracy: 1.0000 - val_loss: 0.8811 - val_accuracy: 0.8608
Epoch 2/5
40/40 [==============================] - 4s 91ms/step - loss: 0.0469 - accuracy: 0.9796 - val_loss: 1.2006 - val_accuracy: 0.8481
Epoch 3/5
40/40 [==============================] - 4s 91ms/step - loss: 0.0131 - accuracy: 0.9963 - val_loss: 0.7225 - val_accuracy: 0.8734
Epoch 4/5
40/40 [==============================] - 4s 91ms/step - loss: 2.3096e-05 - accuracy: 1.0000 - val_loss: 0.6539 - val_accuracy: 0.8734
Epoch 5/5
40/40 [==============================] - 4s 91ms/step - loss: 3.6832e-06 - accuracy: 1.0000 - val_loss: 0.6528 - val_accuracy: 0.8734


In [ ]:
model = load_model('/content/drive/My Drive/CV992/model_h.h5')

In [ ]:
_, acc = model.evaluate(X_train, y_train)
print('Train Accuracy: %.2f' % (acc*100))
_, acc = model.evaluate(X_test, y_test)
print('Test Accuracy: %.2f' % (acc*100))

20/20 [==============================] - 3s 140ms/step - loss: 4.0292e-06 - accuracy: 1.0000
Train Accuracy: 100.00
3/3 [==============================] - 0s 104ms/step - loss: 0.8927 - accuracy: 0.8481
Test Accuracy: 84.81


In [ ]:
def car_truck_classifier(address):
    answer = None
    try:
        img= Image.open(address).convert('RGB')
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            prediction = model.predict(np.array([img]))[0][0]
            answer = 'Car' if round(prediction) == 0 else 'truck'
            answer = 'it is a ' + answer + '!'
    except:
        print("ignored loading", each)
    return answer

In [ ]:
test_img_address = 'download.jpg'
print(car_truck_classifier(test_img_address))

it is a Car!
